In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from google.colab import files
from google.colab import drive
import pandas as pd 

In [ ]:
# Open a file upload dialog
# Select here all files to upload!
# If already uploaded, just press 'Cancel Upload'
uploaded = files.upload()

In [ ]:
# Set the path to the data
# On local machine use the relative path, for example
# path = 'NLP labelled data preview/english set/'
# On Google Colab use this path
# '/content/'
path = '/content/'

In [ ]:
# Load the dataset
file = 'High_Confidence_Comments_Original_English.csv'
all_comments = pd.read_csv(path + file)

In [ ]:
# Load Tokenizer and model 
tokenizer = PegasusTokenizer.from_pretrained('tuner007/pegasus_paraphrase')
model = PegasusForConditionalGeneration.from_pretrained('tuner007/pegasus_paraphrase')

In [ ]:
# Try to use GPU 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

In [ ]:
def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
# Function to generate paraphrases and expand DataFrame
# We want to add the paraphrased comments right under the original comment, for each comment respectively
def expand_comments(df, num_return_sequences, num_beams):
    new_rows = []
    
    for index, row in df.iterrows():
        original_comment = row['Comment']
        label = row['Label']  # Change 'labels' to 'Label'
        new_rows.append({'Comment': original_comment, 'Label': label})  # Change 'labels' to 'Label'
        
        generated_comments = get_response(original_comment, num_return_sequences, num_beams)
        
        for gen_comment in generated_comments:
            new_rows.append({'Comment': gen_comment, 'Label': label})  # Change 'labels' to 'Label'
    
    expanded_df = pd.DataFrame(new_rows)
    expanded_df.reset_index(drop=True, inplace=True)
    return expanded_df

In [ ]:
num_beams = 10 # parameter for beam search, a search strategy used to generate sequences in language generation tasks
num_return_sequences = 10 # How many different sequences to generate

comments_augmented = expand_comments(all_comments, num_return_sequences, num_beams)



In [ ]:
comments_augmented.to_csv(path + 'High_Confidence_Comments_Original_English_Augmented.csv', index=True)


In [ ]:
# Download the file to your local machine (from google colab)
files.download(path + 'High_Confidence_Comments_Original_English_Augmented.csv')